In [1]:
EXPORT_DIR = "../survey/import_new/"

In [2]:
from gpt2outputdataset.detector_radford import DetectorRadford
from detectgpt.detector_detectgpt import DetectorDetectGPT
from detector_guo import DetectorGuo
from explainer_wrappers import LIME_Explainer, SHAP_Explainer, Anchor_Explainer

In [3]:
explainer_classes = [LIME_Explainer, SHAP_Explainer, Anchor_Explainer]
detector_classes = [ DetectorGuo, DetectorRadford, DetectorDetectGPT]

In [4]:
# explainer_classes = [Anchor_Explainer ]
# detector_classes = [ DetectorGuo]

In [5]:
import pandas as pd
import json
import os

In [6]:
df = pd.read_pickle("./dataset_user_study_new.pkl")


In [7]:
df.groupby(["Detector"]).count()

,Explainer,Documents Phases 1+3,Documents Phases 2+4,f(a),f(b),GT a,GT b,idx a,idx b,Spacy Similarity,Jaccard Similarity,Cosine Similarity tfidf,hash a,hash b
Detector,,,,,,,,,,,,,,
DetectorDetectGPT,18,18,18,18,18,18,18,18,18,18,18,18,18,18
DetectorGuo,18,18,18,18,18,18,18,18,18,18,18,18,18,18
DetectorRadford,18,18,18,18,18,18,18,18,18,18,18,18,18,18


In [8]:
if not os.path.exists(os.path.join(EXPORT_DIR)): 
    os.makedirs(os.path.join(EXPORT_DIR))
if not os.path.exists(os.path.join(EXPORT_DIR, "explanations")): 
    os.makedirs(os.path.join(EXPORT_DIR, "explanations"))
if not os.path.exists(os.path.join(EXPORT_DIR, "explanations", "html")): 
    os.makedirs(os.path.join(EXPORT_DIR, "explanations", "html"))
if not os.path.exists(os.path.join(EXPORT_DIR, "explanations", "data")): 
    os.makedirs(os.path.join(EXPORT_DIR, "explanations", "data"))

In [9]:
from IPython.core.display import display, HTML
from tqdm import tqdm

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display


While the explanations are cached, this will still take some time as `detector.predict_proba` is called.

In [10]:
df.groupby("Detector").count()

,Explainer,Documents Phases 1+3,Documents Phases 2+4,f(a),f(b),GT a,GT b,idx a,idx b,Spacy Similarity,Jaccard Similarity,Cosine Similarity tfidf,hash a,hash b
Detector,,,,,,,,,,,,,,
DetectorDetectGPT,18,18,18,18,18,18,18,18,18,18,18,18,18,18
DetectorGuo,18,18,18,18,18,18,18,18,18,18,18,18,18,18
DetectorRadford,18,18,18,18,18,18,18,18,18,18,18,18,18,18


In [11]:

for detector_name, group in df.groupby("Detector"):
    if detector_name == "DetectorDetectGPT":
        detector = DetectorDetectGPT()
    if detector_name == "DetectorRadford":
        detector = DetectorRadford()
    if detector_name == "DetectorGuo":
        detector = DetectorGuo()

    documents_group = list(group.reset_index().iterrows()) # reset_index: count from 0 in each group
    
    for explainer_class in explainer_classes:
        explainer = explainer_class(detector)
        for index, row in tqdm(documents_group, "Exporting " + detector.__class__.__name__ + " " + explainer.__class__.__name__):
            a = row["Documents Phases 1+3"]
            b = row["Documents Phases 2+4"]
            # Phase 1 + 3: 
            path_explanation_html = os.path.join(EXPORT_DIR, "explanations", "html", explainer.get_hash(a)+".html")
            with open(path_explanation_html, "w", encoding="UTF-8") as text_file:
                text_file.write(explainer.get_highlighted_text_HTML(a))
            #  display(HTML(explainer.get_highlighted_text_HTML(a)))
            path_explanation_json = os.path.join(EXPORT_DIR, "explanations", "data", explainer.get_hash(a)+".json")
            with open(path_explanation_json, "w", encoding="UTF-8") as text_file:
                explanation_data = { # do not include explanation
                    "document_nr": index,
                    "detector" : detector.__class__.__name__,
                    "explainer" : explainer.__class__.__name__,
                    "ground_truth" : row["GT a"],
                    "detector_label" : int(detector.predict_label([row["Documents Phases 1+3"]])[0]),
                    "detector_p_machine": float(detector.predict_proba([row["Documents Phases 1+3"]])[0][0]),
                    "detector_p_human": float(detector.predict_proba([row["Documents Phases 1+3"]])[0][1]),
                    "document": row["Documents Phases 1+3"],
                    "explanation_filename": explainer.get_hash(a),               
                }
                text_file.write(json.dumps(explanation_data))
            # Phase 2+4: Just string
            path_explanation_html = os.path.join(EXPORT_DIR, "explanations", "html", explainer.get_hash(b)+".html")
            #  display(HTML(explainer.get_highlighted_text_HTML(b)))
            # with open(path_explanation_html, "w", encoding="UTF-8") as text_file:
            #     text_file.write(explainer.get_highlighted_text_HTML(b))
            path_explanation_json = os.path.join(EXPORT_DIR, "explanations", "data", explainer.get_hash(b)+".json")
            with open(path_explanation_json, "w", encoding="UTF-8") as text_file:
                explanation_data = { # do not include explanation
                    "document_nr": index,
                    "detector" : detector.__class__.__name__,
                    "explainer" : explainer.__class__.__name__,
                    "document": row["Documents Phases 2+4"]                
                }
                text_file.write(json.dumps(explanation_data))
        

Using cache dir ./.cache
Loading BASE model EleutherAI/pythia-70m...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


MOVING BASE MODEL TO GPU...DONE (0.18s)
DONE (0.08s)


Exporting DetectorDetectGPT Anchor_Explainer: 100%|██████████| 18/18 [00:49<00:00,  2.77s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Exporting DetectorRadford Anchor_Explainer: 100%|██████████| 18/18 [00:02<00:00,  7.08it/s]
